In [1]:

import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from keras.layers import *

print(tf.__version__)


1.12.0


Using TensorFlow backend.


In [2]:

def ConvBnAct(x, n_filters=64, kernel=(2,2), strides=(1,1), padding='valid'):
  
  
        filters = n_filters
        kernel = kernel
        strides = strides
        activation = tf.nn.relu
        
        #print('padding', padding)
        conv_ =  Conv2D(filters=filters,
                      kernel_size = kernel,
                      strides = strides, 
                      padding=padding)

        batch_norm = BatchNormalization()

        activation = Activation(activation)
    

        x = conv_(x) 
        x = batch_norm(x)

        x = activation(x)

        return x

In [3]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
t = np.random.randn(10, 256, 256, 64)
block = ConvBnAct(norm)



In [4]:
def ConvAct( x, n_filters, kernel=(1,1), activation = tf.nn.relu, pooling=False):
  


      poolingLayer = AveragePooling2D(pool_size=(1,1), padding='same')
      convLayer = Conv2D(filters = n_filters,
                         kernel_size = kernel,
                         strides=1)

      activation = Activation(activation)

      if pooling:
        x = poolingLayer(x)

      x = convLayer(x)
      x = activation(x)

      return x

In [5]:

norm = tf.random_normal([10, 127, 127, 64], mean=-1, stddev=4)

block = ConvAct(norm, n_filters=64)
print(block)
print(block.shape)

Tensor("activation_2/Relu:0", shape=(10, 127, 127, 64), dtype=float32)
(10, 127, 127, 64)


In [6]:
def AttentionRefinmentModule(inputs, n_filters):
  
    filters = n_filters
    
    poolingLayer = AveragePooling2D(pool_size = (1,1), padding='same')
    
    x = poolingLayer(inputs)
    
    x = ConvBnAct(x, kernel = (1,1), n_filters = filters)
    
    print(x.shape)
    print(inputs.shape)
    
    return multiply([inputs,x])

In [7]:
norm = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = AttentionRefinmentModule(norm, n_filters=64)

print(block.shape)

(10, 256, 256, 64)
(10, 256, 256, 64)
(10, 256, 256, 64)


In [8]:

def FeatureFusionModule(input_f, input_s,n_filters):
#     self.conv_bn_act = ConvBnAct(n_filters=n_filters, kernel=(3, 3), padding='same')
#     self.conv_act1 = ConvAct(n_filters=n_filters, pooling=True)
#     self.conv_act2 = ConvAct(n_filters=n_filters, pooling=False, activation = tf.nn.sigmoid)
    activation = tf.nn.sigmoid
    
       
    concate = Concatenate(axis=-1)([input_f, input_s])
    
    branch0 =  ConvBnAct(concate, n_filters=n_filters, kernel=(3, 3), padding='same') 
    branch_1  = ConvAct(branch0, n_filters=n_filters, pooling=True)
    branch_1 = ConvAct(branch_1, n_filters=n_filters, pooling=False, activation = tf.nn.sigmoid)
    
    x = multiply([branch0, branch_1])
    return  Add()([branch0, x])


In [9]:
norm1 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)
norm2 = tf.random_normal([10, 256, 256, 64], mean=-1, stddev=4)

block = FeatureFusionModule(norm1, norm2, n_filters=64)

print(block.shape)

(10, 256, 256, 64)


In [10]:

def ContextPath(layer_13, layer_14):
    
    globalmax = GlobalAveragePooling2D()
    upsample_1 = UpSampling2D(size=(2, 2))
    upsample_2 = UpSampling2D(size=(2, 2))
    upsample_3 = UpSampling2D(size=(2, 2))
    upsample_4 = UpSampling2D(size=(2, 2))

    globalmax = globalmax(layer_14)
    upsample1 = upsample_1(layer_14)
    first_ = Add()([globalmax, upsample1])
    
    seg1 = AttentionRefinmentModule(layer_13, n_filters=1024)
    seg2 = AttentionRefinmentModule(layer_14, n_filters=2048)
    
    up_3 = upsample_2(seg1)
    up_4 = upsample_3(seg2)

    
    cnc = Concatenate(axis=-1)([up_3, up_4])
    cnc = Concatenate(axis=-1)([cnc, first_])


    context_features = upsample_4(cnc)
    return context_features

In [11]:

layer_13 = tf.random_normal([10, 7, 7, 1024], mean=-1, stddev=4)
layer_14 = tf.random_normal([10, 7, 7, 2048], mean=-1, stddev=4)

block = ContextPath(layer_13,layer_14)

print(block.shape)

(10, 7, 7, 1024)
(10, 7, 7, 1024)
(10, 7, 7, 2048)
(10, 7, 7, 2048)
(10, 28, 28, 5120)


In [12]:
def FinalModel(x, layer_13, layer_14):

    
    upsample = UpSampling2D(size=(8, 8))
   # self.feature_fusion = FeatureFusionModule(32)

    #spatial path
    x = ConvBnAct(x, 32, strides=2)
    x = ConvBnAct(x, 64, strides=2)
    x = ConvBnAct(x, 156, strides=2)
    
    #context path
    cp = ContextPath(layer_13, layer_14)
    
   
    print('fusion')
    fusion = FeatureFusionModule(cp, x, 32)
    print('fusion')
    ans = upsample(fusion)
    
    ans = Activation(tf.nn.softmax)(ans)
    
    return ans


In [13]:
from keras.applications import *
import tensorflow as tf
Xception_model = Xception(weights='imagenet',input_shape= (224,224,3), include_top=False)

inputs = Input(shape=(224,224,3))
x = Lambda(lambda x: x, input_shape=(224,224,3), output_shape=(224, 224, 3))


In [14]:
Xception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [15]:
layer_13 = Xception_model.get_layer('block13_pool').output
layer_14 = Xception_model.output

print(layer_13.shape)
print(layer_14.shape)

(?, 7, 7, 1024)
(?, 7, 7, 2048)


In [16]:
# # 16x Down
layer_13 = Xception_model.get_layer('block13_pool').output
layer_14 = Xception_model.output


In [17]:
# for layer in model.layers[:5]:
#     layer.trainable = False
input_layer = tf.random_normal([10, 224, 224, 3], mean=-1, stddev=4)

layer_13 = tf.random_normal([10, 7, 7, 1024], mean=-1, stddev=4)
layer_14 = tf.random_normal([10, 7, 7, 2048], mean=-1, stddev=4)

block = FinalModel(input_layer, layer_13, layer_14)

print(block.shape)

(10, 7, 7, 1024)
(10, 7, 7, 1024)
(10, 7, 7, 2048)
(10, 7, 7, 2048)
fusion
fusion
(10, 224, 224, 32)


In [18]:
from keras.applications.xception import Xception,preprocess_input
from keras.utils import to_categorical


In [19]:
import os 
import numpy as np 
from keras.preprocessing import image
import cv2

PATH = os.getcwd()
train_path = PATH+'\data\\train\\'
trainy_path = PATH+'\data\\train_labels\\'

val_path = PATH+'\data\\val\\'
valy_path = PATH+'\data\\val_labels\\'

print(train_path)

train_batch = os.listdir(train_path)
trainy_batch = os.listdir(trainy_path)

val_batch = os.listdir(val_path)
valy_batch = os.listdir(valy_path)

X_train = []
y_train = []

X_val = []
y_val = []

# if data are in form of images
for sample in train_batch:
    img_path = train_path+sample
    x = cv2.imread(img_path)
    # preprocessing if required
    x = cv2.resize(x, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

    X_train.append(x)


for sample in trainy_batch:
    img_path = trainy_path+sample
    x = cv2.imread(img_path)
    # preprocessing if required
    x = cv2.resize(x, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

    y_train.append(x)
    


    
# if data are in form of images
for sample in val_batch:
    img_path = val_path+sample
    x = cv2.imread(img_path)
    # preprocessing if required
    x = cv2.resize(x, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

    X_val.append(x)


for sample in valy_batch:
    img_path = valy_path+sample
    x = cv2.imread(img_path)
    # preprocessing if required
    x = cv2.resize(x, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)

    y_val.append(x)



C:\Users\Kiko-PC\Documents\Image-Segmentation\data\train\


In [20]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [21]:
X_val = np.array(X_val)
y_val = np.array(y_val)

In [22]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(421, 224, 224, 3) (421, 224, 224, 3) (112, 224, 224, 3) (112, 224, 224, 3)


In [23]:

def get_label_info(csv_path):
    """
    Retrieve the class names and label values for the selected dataset.
    Must be in CSV format!

    # Arguments
        csv_path: The file path of the class dictionairy
        
    # Returns
        Two lists: one for the class names and the other for the label values
    """
    filename, file_extension = os.path.splitext(csv_path)
    if not file_extension == ".csv":
        return ValueError("File is not a CSV!")

    class_names = []
    label_values = []
    with open(csv_path, 'r') as csvfile:
        file_reader = csv.reader(csvfile, delimiter=',')
        header = next(file_reader)
        for row in file_reader:
            class_names.append(row[0])
            label_values.append([int(row[1]), int(row[2]), int(row[3])])
        # print(class_dict)
    return class_names, label_values

def one_hot_it(label, label_values):
    """
    Convert a segmentation image label array to one-hot format
    by replacing each pixel value with a vector of length num_classes

    # Arguments
        label: The 2D array segmentation image label
        label_values
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of num_classes
    """
    # st = time.time()
    # w = label.shape[0]
    # h = label.shape[1]
    # num_classes = len(class_dict)
    # x = np.zeros([w,h,num_classes])
    # unique_labels = sortedlist((class_dict.values()))
    # for i in range(0, w):
    #     for j in range(0, h):
    #         index = unique_labels.index(list(label[i][j][:]))
    #         x[i,j,index]=1
    # print("Time 1 = ", time.time() - st)

    # st = time.time()
    # https://stackoverflow.com/questions/46903885/map-rgb-semantic-maps-to-one-hot-encodings-and-vice-versa-in-tensorflow
    # https://stackoverflow.com/questions/14859458/how-to-check-if-all-values-in-the-columns-of-a-numpy-matrix-are-the-same
    semantic_map = []
    for colour in label_values:
        # colour_map = np.full((label.shape[0], label.shape[1], label.shape[2]), colour, dtype=int)
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)
    # print("Time 2 = ", time.time() - st)

    return semantic_map
    
def reverse_one_hot(image):
    """
    Transform a 2D array in one-hot format (depth is num_classes),
    to a 2D array with only 1 channel, where each pixel value is
    the classified class key.

    # Arguments
        image: The one-hot format image 
        
    # Returns
        A 2D array with the same width and hieght as the input, but
        with a depth size of 1, where each pixel value is the classified 
        class key.
    """
    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,1])

    # for i in range(0, w):
    #     for j in range(0, h):
    #         index, value = max(enumerate(image[i, j, :]), key=operator.itemgetter(1))
    #         x[i, j] = index

    x = np.argmax(image, axis = -1)
    return x


def colour_code_segmentation(image, label_values):
    """
    Given a 1-channel array of class keys, colour code the segmentation results.

    # Arguments
        image: single channel array where each value represents the class key.
        label_values
        
    # Returns
        Colour coded image for segmentation visualization
    """

    # w = image.shape[0]
    # h = image.shape[1]
    # x = np.zeros([w,h,3])
    # colour_codes = label_values
    # for i in range(0, w):
    #     for j in range(0, h):
    #         x[i, j, :] = colour_codes[int(image[i, j])]
    
    colour_codes = np.array(label_values)
    x = colour_codes[image.astype(int)]

    return x


In [24]:
import csv
class_names_list, label_values =get_label_info("data/class_dict.csv")
class_names_string = ""
for class_name in class_names_list:
    if not class_name == class_names_list[-1]:
        class_names_string = class_names_string + class_name + ", "
    else:
        class_names_string = class_names_string + class_name

num_classes = len(label_values)

In [25]:
print(num_classes)

32


In [26]:
# y_train = np.float32(one_hot_it(y_train[:200], label_values=label_values))



In [27]:
print(y_train.shape)

(421, 224, 224, 3)


In [42]:

def train_generator(samplesX, samplesY, label_values, batch_size=16, is_training=True):
    """
    Lazy batch train/validation generator for memory efficiency
    """
    num_samples = samplesX.shape[0]

    print(samplesX.shape)
    print(samplesY.shape)
    while 1: # Loop forever so the generator never terminates
        #shuffle the samples once the whole data is processed into batches
        #split data into batches
        for offset in range(0, num_samples, batch_size):
            X_train = samplesX[offset:offset+batch_size]
            y_train = samplesY[offset:offset+batch_size]
                    
            
            y_train = np.float32(one_hot_it(y_train, label_values=label_values))
            
            print('--')
            print(X_train.shape)
            print(y_train.shape)
            
            yield ([X_train, X_train], y_train)

In [43]:
from keras.models import Model

In [44]:
xception =  Xception(weights='imagenet',input_shape= (224,224,3), include_top=False)

inputs = Input(shape=(224,224,3))

    
#spatial path



layer_13 = xception.get_layer('block13_pool').output
layer_14 = xception.output

ans = FinalModel(inputs, layer_13, layer_14)





(?, 7, 7, 1024)
(?, 7, 7, 1024)
(?, 7, 7, 2048)
(?, 7, 7, 2048)
fusion
fusion


In [45]:

# for inx, layer in enumerate(xception.layers):
#     if inx<100:
#         layer.trainable=False
#     else:
#         layer.trainable = True

In [46]:
print(x.shape)

(224, 224, 3)


In [47]:
model = Model(inputs = [inputs, xception.input], output = [ans])

C:\Users\Kiko-PC\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


In [48]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [49]:
print(xception.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [50]:
# X_train = tf.convert_to_tensor(
#     X_train[:10],

# )
# y_train = tf.convert_to_tensor(
#     y_train[:10],

# )

In [51]:
BATCH_SIZE = 16
EPOCHS = 1000

from keras.callbacks import ModelCheckpoint



#model.compile(optimizer=tf.train.RMSPropOptimizer(learning_rate=0.0001, decay=0.99), 

model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.005), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])


#Get model training checkpoints
checkpoint = ModelCheckpoint('checkpoints/model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=False,
                                 mode='auto')


#model = load_model('checkpoints/model.h5')
#generator = train_generator(X_train, y_train, label_values)

                                   

In [52]:
generator = train_generator(X_train, y_train, label_values)
validation_generator = train_generator(X_val, y_val, label_values)


In [53]:
print(len(xception.layers))

132


In [54]:
history_object = model.fit_generator(generator,
                                     X_train.shape[0]//BATCH_SIZE,
                                     epochs=EPOCHS, 
                                     validation_data=validation_generator,
                                     validation_steps = X_val.shape[0]//BATCH_SIZE,
                                     shuffle=True)

(421, 224, 224, 3)
(421, 224, 224, 3)
Epoch 1/1000
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
 3/26 [==>...........................] - ETA: 1:20 - loss: 1.4073 - acc: 0.1562--
(16, 224, 224, 3)
(16, 224, 224, 32)
 5/26 [====>.........................] - ETA: 47s - loss: 1.2873 - acc: 0.2231--
(16, 224, 224, 3)
(16, 224, 224, 32)
 7/26 [=======>......................] - ETA: 32s - loss: 1.2340 - acc: 0.2793--
(16, 224, 224, 3)
(16, 224, 224, 32)
 9/26 [=========>....................] - ETA: 23s - loss: 1.1895 - acc: 0.3121--
(16, 224, 224, 3)
(16, 224, 224, 32)
11/26 [===========>.................

--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
26/26 [==============================] - 17s 641ms/step - loss: 0.3843 - acc: 0.4150 - val_loss: 0.7599 - val_acc: 0.3848
Epoch 4/1000
 1/26 [>.............................] - ETA: 9s - loss: 0.2986 - acc: 0.4729--
(16, 224, 224, 3)
(16, 224, 224, 32)
 4/26 [===>..........................] - ETA: 6s - loss: 0.3338 - acc: 0.4340--
(16, 224, 224, 3)
(16, 224, 224, 32)
 6/26 [=====>........................] - ETA: 5s - loss: 0.3339 - acc: 0.4175--
(16, 224, 224, 3)
(16, 224, 224, 32)
 8/26 [========>.....................] - ETA: 5s - loss: 0.3453 - acc: 0.4124--
(16, 224, 224, 3)
(16, 224, 224, 32)
10/26 [==========>...................] - ETA: 4s - loss: 0.3452 - acc: 0.4333--
(16, 224, 224, 3)
(16, 224, 224, 32)
12/26 [============>.................] -

--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(5, 224, 224, 3)
(5, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
26/26 [==============================] - 17s 640ms/step - loss: 0.2430 - acc: 0.4340 - val_loss: 0.5628 - val_acc: 0.3938
Epoch 7/1000
 1/26 [>.............................] - ETA: 8s - loss: 0.1695 - acc: 0.3713--
(16, 224, 224, 3)
(16, 224, 224, 32)
 3/26 [==>...........................] - ETA: 7s - loss: 0.2070 - acc: 0.3530--
(16, 224, 224, 3)
(16, 224, 224, 32)
 6/26 [=====>........................] - ETA: 5s - loss: 0.1931 - acc: 0.4125--
(16, 224, 224, 3)
(16, 224, 224, 32)
 8/26 [========>.....................] - ETA: 5s - loss: 0.2069 - acc: 0.408

--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(5, 224, 224, 3)
(5, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
26/26 [==============================] - 17s 660ms/step - loss: 0.1748 - acc: 0.4402 - val_loss: 0.4850 - val_acc: 0.3964
Epoch 10/1000
 1/26 [>.............................] - ETA: 7s - loss: 0.1612 - acc: 0.4374--
(16, 224, 224, 3)
(16, 224, 224, 32)
 3/26 [==>...........................] - ETA: 7s - loss: 0.1567 - acc: 0.4465--
(16, 224, 224, 3)
(16, 224, 2

25/26 [===========================>..] - ETA: 0s - loss: 0.1422 - acc: 0.4476--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
26/26 [==============================] - 17s 646ms/step - loss: 0.1420 - acc: 0.4473 - val_loss: 0.2766 - val_acc: 0.4399
Epoch 13/1000
 1/26 [>.............................] - ETA: 8s - loss: 0.1353 - acc: 0.3932
(16, 224, 224, 3)
(16, 224, 2

...

 3/26 [==>...........................] - ETA: 7s - loss: 0.0820 - acc: 0.4281--
(16, 224, 224, 3)
(16, 224, 224, 32)
 6/26 [=====>........................] - ETA: 6s - loss: 0.0924 - acc: 0.4109--
(16, 224, 224, 3)
(16, 224, 224, 32)
 8/26 [========>.....................] - ETA: 5s - loss: 0.0912 - acc: 0.4153--
(16, 224, 224, 3)
(16, 224, 224, 32)
11/26 [===========>..................] - ETA: 4s - loss: 0.0920 - acc: 0.4272--
(16, 224, 224, 3)
(16, 224, 224, 32)
13/26 [==============>...............] - ETA: 3s - loss: 0.0921 - acc: 0.4271--
(16, 224, 224, 3)
(16, 224, 224, 32)
15/26 [================>.............] - ETA: 3s - loss: 0.0929 - acc: 0.4291--
(16, 224, 224, 3)
(16, 224, 224, 32)
16/26 [=================>............] - ETA: 3s - loss: 0.0922 - acc: 0.4393--
(16, 224, 224, 3)
(16, 224, 224, 32)
17/26 [==================>...........] - ETA: 3s - loss: 0.0918 - acc: 0.4415--
(16, 224, 224, 3)
(16, 224, 224, 32)
18/26 [===================>..........] - ETA: 2s - loss: 0.0919 

Epoch 176/1000
--
(16, 224, 224, 3)
(16, 224, 224, 32)
 3/26 [==>...........................] - ETA: 7s - loss: 0.0707 - acc: 0.4022--
(16, 224, 224, 3)
(16, 224, 224, 32)
 5/26 [====>.........................] - ETA: 6s - loss: 0.0639 - acc: 0.4059--
(16, 224, 224, 3)
(16, 224, 224, 32)
 7/26 [=======>......................] - ETA: 5s - loss: 0.0686 - acc: 0.4210--
(16, 224, 224, 3)
(16, 224, 224, 32)
 8/26 [========>.....................] - ETA: 5s - loss: 0.0689 - acc: 0.4172--
(5, 224, 224, 3)
(5, 224, 224, 32)
10/26 [==========>...................] - ETA: 5s - loss: 0.0740 - acc: 0.4075--
(16, 224, 224, 3)
(16, 224, 224, 32)
13/26 [==============>...............] - ETA: 3s - loss: 0.0707 - acc: 0.4264--
(16, 224, 224, 3)
(16, 224, 224, 32)
15/26 [================>.............] - ETA: 3s - loss: 0.0738 - acc: 0.4274--
(16, 224, 224, 3)
(16, 224, 224, 32)
16/26 [=================>............] - ETA: 3s - loss: 0.0741 - acc: 0.4267--
(16, 224, 224, 3)
(16, 224, 224, 32)
17/26 [====

--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
26/26 [==============================] - 17s 658ms/step - loss: 0.0676 - acc: 0.4557 - val_loss: 0.1677 - val_acc: 0.4454
Epoch 179/1000
--
(16, 224, 224, 3)
(16, 224, 224, 32)
 3/26 [==>...........................] - ETA: 7s - loss: 0.0559 - acc: 0.4779--
(16, 224, 224, 3)
(16, 224, 224, 32)
 5/26 [====>.........................] - ETA: 6s - loss: 0.0590 - acc: 0.4584--
(16, 224, 224, 3)
(16, 224, 224, 32)
 7/26 [=======>......................] - ETA: 6s - loss: 0.0571 - acc: 0.4312--
(16, 224, 224, 3)
(16, 224, 224, 32)
 9/26 [=========>....................] - ETA: 5s - loss: 0.0573 - acc: 0.4392--
(16, 224, 224, 3)
(16, 224, 224, 32)
11/26 [===========>..................] - ETA: 4s - loss: 0.0578 - acc: 0.4357--
(16, 224, 224, 3)
(16, 224, 224, 32

--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
--
(16, 224, 224, 3)
(16, 224, 224, 32)
26/26 [==============================] - 17s 643ms/step - loss: 0.0601 - acc: 0.4552 - val_loss: 0.1611 - val_acc: 0.4478
Epoch 182/1000
 1/26 [>.............................] - ETA: 7s - loss: 0.0905 - acc: 0.5830--
(16, 224, 224, 3)
(16, 224, 224, 32)
 3/26 [==>...........................] - ETA: 7s - loss: 0.0753 - acc: 0.5451--
(16, 224, 224, 3)
(16, 224, 224, 32)
 4/26 [===>..........................] - ETA: 6s - loss: 0.0711 - acc: 0.5402--
(16, 224, 224, 3)
(16, 224, 224, 32)


KeyboardInterrupt: 

In [ ]:
# import tensorflow as tf
# model = tf.keras.models.load_model('checkpoints/model-029.h5')
print(X_train[0:1].shape)

In [55]:
t  = model.predict([X_train[0:1], X_train[0:1]])

In [56]:
print(t[0].shape)

(224, 224, 32)


In [57]:
print(np.max(t))

0.9983512


In [58]:
output_image = reverse_one_hot(t)

In [59]:
print(output_image.shape)

(1, 224, 224)


In [60]:
out_vis_image = colour_code_segmentation(output_image, label_values)

print(out_vis_image.shape)

(1, 224, 224, 3)


In [61]:
cv2.imwrite("%s_pred.png"%("kiko"),cv2.cvtColor(np.uint8(out_vis_image[0]), cv2.COLOR_RGB2BGR))

True

In [62]:
cv2.imwrite("%s_pred.png"%("kiko2"),cv2.cvtColor(np.uint8(X_train[0]), cv2.COLOR_RGB2BGR))

True

In [69]:
import cv2

background = cv2.imread('kiko_pred.png') 
overlay = cv2.imread('kiko2_pred.png')

added_image = cv2.addWeighted(background,1,overlay,1,0)

cv2.imwrite('combined.png', added_image)

True